In [49]:
# CREATE Accommodation (
#   ID INT
#   시설 이름 CHAR(100)
#   별점 INT
#   시설 유형 CHAR(100)
#   가격 (INT)
#   위치 CHAR(100)
#   체크인 날짜 DATETIME
#   요일 CHAR(100)
# );

##
#/html/body/div[11]/div[2]/div[3]/button[2] 

In [13]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
import datetime
import json

hotel_elements = []

def scrolling(driver):
    for i in range(0,27493,700):
        driver.execute_script(f"window.scrollTo(0,{i});")
        time.sleep(1)


def crawl_and_extract(loc_satis,save_path):
    weekday = { 0:'MON',
                1:'TUE',
                2:'WED',
                3:'THU',
                4:'FRI',
                5:'SAT',
                6:'SUN'
            }

    year_list = [2023]
    month_list = [9]
    # day_list = [i for i in range(1,30)]
    day_list = [30]

    def deserialize_datetime(json_obj):
        if 'datetime' in json_obj:
            return datetime.fromisoformat(json_obj['datetime'])
        return json_obj

    # Load the data from the JSON file with custom deserialization function
    with open(save_path, 'r') as file:
        loaded_data_list = json.load(file, object_hook=deserialize_datetime)

    # Print the loaded data
    # print(len(loaded_data_list))

    extracted_data = loaded_data_list
    except_popup=0
    except_xpath=0
    for year in year_list:
        for month in month_list:
            init_datetime = datetime.datetime(year, month, 1)
            # Get the year, week number, and weekday using isocalendar()
            year, init_week_number, _ = init_datetime.isocalendar()
            for day in day_list:
                start_datetime = datetime.datetime(year,month,day)
                try:
                    end_datetime = datetime.datetime(year,month,day+1)

                    year, startdate_week_number, _ = start_datetime.isocalendar()
                    startdate_week_number = startdate_week_number - init_week_number + 1
                    startdate_weekday = start_datetime .weekday()
                    year, enddate_week_number, _ = end_datetime.isocalendar()
                    enddate_week_number = enddate_week_number - init_week_number + 1
                    enddate_weekday = end_datetime .weekday()
                except:
                    end_datetime = datetime.datetime(year,month+1,1)

                    year, startdate_week_number, _ = start_datetime.isocalendar()
                    startdate_week_number = startdate_week_number - init_week_number + 1
                    startdate_weekday = start_datetime .weekday()
                    enddate_week_number = 1
                    enddate_weekday = end_datetime .weekday()
                # Chrome 브라우저를 사용하여 웹 드라이버 시작
                driver = webdriver.Chrome()

                # Agoda 사이트로 이동
                driver.get("https://www.agoda.com/ko-kr/")

                # 페이지가 로딩될 때까지 잠시 기다립니다.
                time.sleep(8)

                # 팝업창 클릭해서 없애기
                try:
                    popup_box = driver.find_element(By.XPATH,"/html/body/div[12]/div[2]/button")
                except:
                    try:
                        popup_box = driver.find_element(By.XPATH,"/html/body/div[11]/div[2]/div[3]/button[2]")
                    except:
                        except_popup+=1
                        continue 
                popup_box.click()

                time.sleep(3)

                # 검색어 입력란을 찾고 "로스엔젤레스"라는 검색어를 입력합니다.

                search_box = driver.find_element(By.XPATH,'//*[@id="textInput"]')
                search_box.send_keys("로스엔젤레스")
                search_box.send_keys(Keys.RETURN)
                time.sleep(4)
                # 검색어 자동 완성에서 해당 지역을 클릭
                
                autocomplete_tab = driver.find_element(By.XPATH,'//*[@id="SearchBoxContainer"]/div[1]/div/div[2]/div/div/div[6]/div/div/ul/li[1]')
                autocomplete_tab.click()
                time.sleep(3)

                # 달력에서 다음 버튼 클릭
                next_month_button = driver.find_element(By.XPATH,'//*[@id="SearchBoxContainer"]/div[1]/div/div[2]/div/div/div[6]/div/div/div[1]/div/div[1]/span[2]')
                next_month_button.click()
                time.sleep(3)

                # 날짜 선택 및 클릭
                start_date_XPATH = f'//*[@id="SearchBoxContainer"]/div[1]/div/div[2]/div/div/div[6]/div/div/div[1]/div/div[2]/div[1]/div[3]/div[{startdate_week_number}]/div[{startdate_weekday+1}]'
                start_date_picker = driver.find_element(By.XPATH,start_date_XPATH)
                start_date_picker.click()
                time.sleep(1)
                if enddate_week_number - startdate_week_number <0:
                    end_date_XPATH = f'//*[@id="SearchBoxContainer"]/div[1]/div/div[2]/div/div/div[6]/div/div/div[1]/div/div[2]/div[2]/div[3]/div[{enddate_week_number}]/div[{enddate_weekday+1}]'
                    end_date_picker = driver.find_element(By.XPATH,end_date_XPATH)
                    end_date_picker.click()
                    time.sleep(1)
                else:
                    end_date_XPATH = f'//*[@id="SearchBoxContainer"]/div[1]/div/div[2]/div/div/div[6]/div/div/div[1]/div/div[2]/div[1]/div[3]/div[{enddate_week_number}]/div[{enddate_weekday+1}]'
                    end_date_picker = driver.find_element(By.XPATH,end_date_XPATH)
                    end_date_picker.click()
                    time.sleep(1)
                minus_picker = driver.find_element(By.XPATH,'//*[@id="occupancy-selector"]/div/div/div[2]/div[2]/div[1]')
                minus_picker.click()
                time.sleep(1)
                search_button = driver.find_element(By.XPATH,'//*[@id="SearchBoxContainer"]/div[2]/button')
                search_button.click()
                time.sleep(3)
                search_button = driver.find_element(By.XPATH,'//*[@id="SearchBoxContainer"]/div[2]/button')
                search_button.click()
                time.sleep(3)

                # search_box.send_keys(Keys.RETURN)

                # 검색 결과가 로딩될 때까지 잠시 기다립니다.
                time.sleep(8)


                # # 숙소 위치 평가 선택
                if loc_satis == 9 :
                    value9_filter = driver.find_element(By.XPATH,'//*[@id="SideBarLocationFilters"]/div[10]/div[2]/ul/li[1]/span')
                    value9_filter.click()
                    time.sleep(5)
                elif loc_satis == 8 :
                    value8_filter = driver.find_element(By.XPATH,'//*[@id="SideBarLocationFilters"]/div[8]/div[2]/ul/li[2]/span')
                    value8_filter.click()
                    time.sleep(5)
                elif loc_satis == 7 :
                    value7_filter = driver.find_element(By.XPATH,'//*[@id="SideBarLocationFilters"]/div[8]/div[2]/ul/li[3]/span')
                    value7_filter.click()
                    time.sleep(5)
                else:
                    pass
                # value6_filter = driver.find_element(By.XPATH,'//*[@id="SideBarLocationFilters"]/div[8]/div[2]/ul/li[4]/span')
                # value6_filter.click()
                # time.sleep(5)

                # 투숙객 평가 점수 선택
                # customer_eval_filter = driver.find_element(By.XPATH,'//*[@id="SideBarLocationFilters"]/div[8]/div[2]/ul/li[1]/span')
                # customer_eval_filter.click()
                # time.sleep(7)

                while(True):
                    ## 위치 평가가 9일 경우, 스크롤링하여 데이터를 수집할 필요가 없음.
                    if loc_satis != 9 :
                        scrolling(driver)

                    hotel_elements = driver.find_elements(By.XPATH, '//*[@class="PropertyCard PropertyCardItem"]')
                    
                    for hotel_element in hotel_elements:
                        try:
                            hotel_name = hotel_element.find_elements(By.TAG_NAME,'h3')[0].text
                            hotel_location = hotel_element.find_elements(By.TAG_NAME,'span')[5].text
                            hotel_price = hotel_element.find_elements(By.CLASS_NAME,'PropertyCardPrice__Value')[0].text
                            satis_suspects  = hotel_element.find_elements(By.TAG_NAME,'p')
                            satis = None
                            for x in satis_suspects:
                                txt = x.text
                                try:
                                    satis = float(txt)
                                    break
                                except:
                                    continue
                            
                        except:
                            except_xpath+=1
                            continue
                        hotel = {'name':hotel_name,
                                'location':hotel_location,
                                'price':hotel_price,
                                'loc_eval':loc_satis,
                                'satisfaction':satis,
                                'checkin date':start_datetime.isoformat(),
                                'weekday':weekday[startdate_weekday]
                                }
                        extracted_data.append(hotel)
                    break
        

                driver.quit()


    file_path = save_path

    # Save the list to the JSON file
    with open(file_path, 'w') as file:
        json.dump(extracted_data, file)
                
    print(len(extracted_data))
    print('=========== excepted: ',except_xpath)
    print(extracted_data)

    return True


Week number in the month: 35
4


In [14]:
crawl_and_extract(9,'data_9.json')
crawl_and_extract(8,'data_8.json')
crawl_and_extract(7,'data_7.json')



NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="occupancy-selector"]/div/div/div[2]/div[2]/div[1]"}
  (Session info: chrome=115.0.5790.99); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
Backtrace:
	GetHandleVerifier [0x00B9A813+48355]
	(No symbol) [0x00B2C4B1]
	(No symbol) [0x00A35358]
	(No symbol) [0x00A609A5]
	(No symbol) [0x00A60B3B]
	(No symbol) [0x00A8E232]
	(No symbol) [0x00A7A784]
	(No symbol) [0x00A8C922]
	(No symbol) [0x00A7A536]
	(No symbol) [0x00A582DC]
	(No symbol) [0x00A593DD]
	GetHandleVerifier [0x00DFAABD+2539405]
	GetHandleVerifier [0x00E3A78F+2800735]
	GetHandleVerifier [0x00E3456C+2775612]
	GetHandleVerifier [0x00C251E0+616112]
	(No symbol) [0x00B35F8C]
	(No symbol) [0x00B32328]
	(No symbol) [0x00B3240B]
	(No symbol) [0x00B24FF7]
	BaseThreadInitThunk [0x778D00C9+25]
	RtlGetAppContainerNamedObj

In [6]:
file_path = 'data_8.json'

def deserialize_datetime(json_obj):
    if 'datetime' in json_obj:
        return datetime.fromisoformat(json_obj['datetime'])
    return json_obj

# Load the data from the JSON file with custom deserialization function
with open(file_path, 'r') as file:
    loaded_data_list = json.load(file, object_hook=deserialize_datetime)

# Print the loaded data
print(len(loaded_data_list))

1088
